# Salary Prediction and Job Market Analysis
### Create Spark Seesion

In [63]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [64]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

## Loading data from CSV file

In [65]:
df = spark.read.csv("file:///home/talentum/job_market_analysis/final_dataset.csv", header=True, inferSchema=True)
df.show()

+-------+-------+---------+---------+-----------------+-----------------+-------------------+------+--------------------+--------------------+-------------+---------------+--------+---------+
|Min_exp|Max_exp|  Min_sal|  Max_sal|        job_title|         location|               date|rating|        company_name|                role|industry_type|functional_area|emp_type|education|
+-------+-------+---------+---------+-----------------+-----------------+-------------------+------+--------------------+--------------------+-------------+---------------+--------+---------+
|    0.0|    1.0| 280838.0| 896847.0|android developer|          chennai|2023-05-23 00:00:00|   3.8|netaxis it soluti...|mobile app developer|           it|software and qa|fulltime| graduate|
|    5.0|   10.0|1211702.0|9090683.0|android developer|      bhubaneswar|2023-05-17 00:00:00|   4.1|a and d recruitme...|mobile app developer|           it|software and qa|fulltime| graduate|
|    3.0|    7.0| 804581.0|1440585.0|and

In [66]:
df.count()

61020

## Removing Outliers Using Z-score

In [67]:
from pyspark.sql.functions import mean, stddev

In [68]:
mean_exp = df.select(mean('Min_exp')).first()[0]
std_exp = df.select(stddev('Min_exp')).first()[0]

ul_Min_exp = mean_exp + 3 * std_exp
print("Upper Limit for Min_exp:", ul_Min_exp)
ll_Min_exp = mean_exp - 3 * std_exp
print("Lower Limit for Min_exp:", ll_Min_exp)

count_outside_range1 = df.filter((df.Min_exp > ul_Min_exp) | (df.Min_exp < ll_Min_exp)).count()
print(count_outside_range1)

Upper Limit for Min_exp: 10.963985340879805
Lower Limit for Min_exp: -3.6754569895195965
765


In [69]:
mean_exp1 = df.select(mean('Max_exp')).first()[0]
std_exp1 = df.select(stddev('Max_exp')).first()[0]

ul_Max_exp = mean_exp1 + 3 * std_exp1
print("Upper Limit for Max_exp:", ul_Max_exp)
ll_Max_exp = mean_exp1 - 3 * std_exp1
print("Lower Limit for Max_exp:", ll_Max_exp)

count_outside_range2 = df.filter((df.Max_exp > ul_Max_exp) | (df.Max_exp < ll_Max_exp)).count()
print(count_outside_range2)

Upper Limit for Max_exp: 16.640599003159547
Lower Limit for Max_exp: -2.2240470529792775
707


In [70]:
mean_sal = df.select(mean('Min_sal')).first()[0]
std_sal = df.select(stddev('Min_sal')).first()[0]

ul_Min_sal = mean_sal + 3 * std_sal
print("Upper Limit for Min_sal:", ul_Min_sal)
ll_Min_sal = mean_sal - 3 * std_sal
print("Lower Limit for Min_sal:", ll_Min_sal)

count_outside_range3 = df.filter((df.Min_sal > ul_Min_sal) | (df.Min_sal < ll_Min_sal)).count()
print(count_outside_range3)

Upper Limit for Min_sal: 2355966.860132613
Lower Limit for Min_sal: -576471.0076252387
668


In [71]:
mean_sal1 = df.select(mean('Max_sal')).first()[0]
std_sal1 = df.select(stddev('Max_sal')).first()[0]

ul_Max_sal = mean_sal1 + 3 * std_sal1
print("Upper Limit for Max_sal:", ul_Max_sal)
ll_Max_sal = mean_sal1 - 3 * std_sal1
print("Lower Limit for Max_sal:", ll_Max_sal)

count_outside_range4 = df.filter((df.Max_sal > ul_Max_sal) | (df.Max_sal < ll_Max_sal)).count()
print(count_outside_range4)

Upper Limit for Max_sal: 48290128.634865075
Lower Limit for Max_sal: -42698630.21116793
21


In [11]:
# Filter the DataFrame based on the outliers
df = df.filter((df.Min_sal < ul_Min_sal) &
               (df.Min_sal > ll_Min_sal) &
               (df.Max_sal < ul_Max_sal) &
               (df.Max_sal > ll_Max_sal) &
               (df.Max_exp < ul_Max_exp) &
               (df.Max_exp > ll_Max_exp) &
               (df.Min_exp < ul_Min_exp) &
               (df.Min_exp > ll_Min_exp))
#df.show()
df.count()

59737

In [12]:
columns_to_select=["Min_exp","Max_exp","Min_sal","Max_sal","job_title"]
new_df = df.select(columns_to_select)
new_df

DataFrame[Min_exp: double, Max_exp: double, Min_sal: double, Max_sal: double, job_title: string]

## Model Building

### One Hot Encoding

In [13]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

In [14]:
indexer = StringIndexer(inputCol="job_title", outputCol="job_titleIndex")
indexed_df = indexer.fit(new_df).transform(new_df)

In [86]:
encoder = OneHotEncoder(inputCol="job_titleIndex", outputCol="job_title_vec")
encoded_df = encoder.transform(indexed_df)

In [73]:
from pyspark.ml.feature import VectorAssembler

In [87]:
assembler = VectorAssembler(inputCols=["Min_exp", "Max_exp", "job_title_vec"], outputCol="features")
assembler= assembler.transform(encoded_df)
#assembler.show()

In [88]:
assembler_df = assembler.select("features", "Min_sal", "Max_sal", "job_title_vec")
#assembler_df.show()

In [77]:
#train_data.show()

In [78]:
#test_data.show()

## Decision Tree Regressor
### Predicting Minimum Salary

In [76]:
train_data, test_data = assembler_df.randomSplit([0.8, 0.2], seed=42)

In [79]:
from pyspark.ml.regression import DecisionTreeRegressor

In [80]:
dt = DecisionTreeRegressor(featuresCol= "features",
                          labelCol= "Min_sal")

In [81]:
dt_model_min = dt.fit(train_data)
dt_model_min

DecisionTreeRegressionModel (uid=DecisionTreeRegressor_b724c260f8cc) of depth 5 with 63 nodes

In [82]:
pred_min = dt_model_min.transform(test_data)

In [83]:
from pyspark.ml.evaluation import RegressionEvaluator
# Root Mean Squared Error
evaluator_min = RegressionEvaluator(labelCol='Min_sal', metricName='rmse')
rmse_min = evaluator_min.evaluate(pred_min)
print("Root Mean Squared Error (RMSE):", rmse_min)
# R-squared
evaluator_min = RegressionEvaluator(labelCol='Min_sal', metricName='r2')
r2_min = evaluator_min.evaluate(pred_min)
print("R-squared:", r2_min)
# Adjusted R-squared
feature_columns = ['Min_exp','Max_exp','job_title_index']
n = test_data.count()
p = len(feature_columns)
adjusted_r2_min = 1 - ((1 - r2_min) * (n - 1)) / (n - p - 1)
print("Adjusted R-squared:", adjusted_r2_min)
# Mean Absolute Error (MAE)
evaluator_min = RegressionEvaluator(labelCol='Min_sal', metricName='mae')
mae_min = evaluator_min.evaluate(pred_min)
print("Mean Absolute Error (MAE):", mae_min)

Root Mean Squared Error (RMSE): 160620.86541117053
R-squared: 0.8447628828433194
Adjusted R-squared: 0.8447237309852722
Mean Absolute Error (MAE): 63941.41490599566


## Decision Tree Regressor
### To predict Maximum Salary

In [84]:
from pyspark.ml.regression import DecisionTreeRegressor
dt1 = DecisionTreeRegressor(featuresCol= "features",
                          labelCol= "Max_sal")

In [28]:
dt_model_max = dt1.fit(train_data)

In [29]:
dt_model_max

DecisionTreeRegressionModel (uid=DecisionTreeRegressor_e8a314bad3e6) of depth 5 with 61 nodes

In [30]:
pred_max = dt_model_max.transform(test_data)

In [31]:
from pyspark.ml.evaluation import RegressionEvaluator
# Root Mean Squared Error
evaluator_max = RegressionEvaluator(labelCol='Max_sal', metricName='rmse')
rmse_max = evaluator_max.evaluate(pred_max)
print("Root Mean Squared Error (RMSE):", rmse_max)
# R-squared
evaluator_max = RegressionEvaluator(labelCol='Max_sal', metricName='r2')
r2_max = evaluator_max.evaluate(pred_max)
print("R-squared:", r2_max)
# Adjusted R-squared
feature_columns = ['Min_exp','Max_exp','job_title_index']
n = test_data.count()
p = len(feature_columns)
adjusted_r2_max = 1 - ((1 - r2_max) * (n - 1)) / (n - p - 1)
print("Adjusted R-squared:", adjusted_r2_max)
# Mean Absolute Error (MAE)
evaluator_max = RegressionEvaluator(labelCol='Max_sal', predictionCol='prediction', metricName='mae')
mae_max = evaluator_max.evaluate(pred_max)
print("Mean Absolute Error (MAE):", mae_max)

Root Mean Squared Error (RMSE): 1026722.364590288
R-squared: 0.8220122443565894
Adjusted R-squared: 0.8219673546325935
Mean Absolute Error (MAE): 439783.6837687601


## RandomForest Regressor
### To predict Minimum Salary

In [32]:
from pyspark.ml.regression import RandomForestRegressor

In [33]:
rf = RandomForestRegressor(featuresCol= "features",
                          labelCol= "Min_sal", numTrees=100, maxDepth=10, maxBins=32)

In [34]:
rf_model_min = rf.fit(train_data)

In [35]:
rf_model_min

RandomForestRegressionModel (uid=RandomForestRegressor_007ceb2f4c3a) with 100 trees

In [36]:
pred_min1 = rf_model_min.transform(test_data)

In [37]:
# Root Mean Squared Error
evaluator_min1 = RegressionEvaluator(labelCol='Min_sal', metricName='rmse')
rmse_min1 = evaluator_min1.evaluate(pred_min1)
print("Root Mean Squared Error (RMSE):", rmse_min1)

# R-squared
evaluator_min1 = RegressionEvaluator(labelCol='Min_sal', metricName='r2')
r2_min1 = evaluator_min1.evaluate(pred_min1)
print("R-squared:", r2_min1)

# Adjusted R-squared
feature_columns = ['Min_exp','Max_exp','job_title_index']
n = test_data.count()
p = len(feature_columns)
adjusted_r2_min1 = 1 - ((1 - r2_min1) * (n - 1)) / (n - p - 1)
print("Adjusted R-squared:", adjusted_r2_min1)

# Mean Absolute Error (MAE)
evaluator_min1 = RegressionEvaluator(labelCol='Min_sal', metricName='mae')
mae_min1 = evaluator_min1.evaluate(pred_min1)
print("Mean Absolute Error (MAE):", mae_min1)

Root Mean Squared Error (RMSE): 159677.58370815124
R-squared: 0.8465808577829277
Adjusted R-squared: 0.84654216443054
Mean Absolute Error (MAE): 68936.19544843242


## RandomForest Regressor
### To predict Maximum Salary

In [38]:
from pyspark.ml.regression import RandomForestRegressor

In [39]:
rf1 = RandomForestRegressor(featuresCol= "features",labelCol= "Max_sal", numTrees=100, maxDepth=10, maxBins=32)

In [40]:
rf_model_max = rf1.fit(train_data)

In [41]:
rf_model_max

RandomForestRegressionModel (uid=RandomForestRegressor_6259c2e617dd) with 100 trees

In [42]:
pred_max1 = rf_model_max.transform(test_data)

In [43]:
# Root Mean Squared Error
evaluator_max1 = RegressionEvaluator(labelCol='Max_sal', metricName='rmse')
rmse_max1 = evaluator_max1.evaluate(pred_max1)
print("Root Mean Squared Error (RMSE):", rmse_max1)

# R-squared
evaluator_max1 = RegressionEvaluator(labelCol='Max_sal', metricName='r2')
r2_max1 = evaluator_max1.evaluate(pred_max1)
print("R-squared:", r2_max1)

# Adjusted R-squared
feature_columns = ['Min_exp','Max_exp','job_title_index']
n = test_data.count()
p = len(feature_columns)
adjusted_r2_max1 = 1 - ((1 - r2_max1) * (n - 1)) / (n - p - 1)
print("Adjusted R-squared:", adjusted_r2_max1)

# Mean Absolute Error (MAE)
evaluator_max1 = RegressionEvaluator(labelCol='Max_sal', metricName='mae')
mae_max1 = evaluator_max1.evaluate(pred_max1)
print("Mean Absolute Error (MAE):", mae_max1)

Root Mean Squared Error (RMSE): 1074690.6516684105
R-squared: 0.8049926301757542
Adjusted R-squared: 0.8049434479891655
Mean Absolute Error (MAE): 611273.9062345586


# Linear Regression

### Predicting Minimum Salary

In [44]:
from pyspark.ml.regression import LinearRegression

In [45]:
train_data_lr = train_data.select("features", "Min_sal")
test_data_lr = test_data.select("features")
#test_data_lr.show()

In [46]:
lr1 = LinearRegression(featuresCol= "features" ,labelCol= "Min_sal",maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [47]:
lr_model_min = lr1.fit(train_data_lr)

In [48]:
#lr_model_min

In [49]:
pred_lr_min = lr_model_min.transform(test_data_lr)


In [50]:
print("Coefficients: %s" % str(lr_model_min.coefficients))

Coefficients: [122792.51872361587,41416.298824579244,1626.5750366436553,-9641.744706053543,18380.537543321523,-11031.205902297796,0.0,-56528.88026379408,13999.495912249646,11692.295256094467,-3460.887382564536,-15953.873757564334,-9409.211681869348,-45789.35641034676,5430.052505321413,-26549.28213728114,42261.32306726676,12121.64071538482,479.13898373579065,-4023.309427967243,7793.219702964808,5576.444262088067,-3252.280703511038,31367.389897759418,21924.59146155987,-1196.9297263374062,-34748.302824149985]


In [51]:
print("Intercept: %s" % str(lr_model_min.intercept))

Intercept: 138157.8542294028


In [52]:
trainingSummary = lr_model_min.summary

In [53]:
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)

RMSE: 172343.634616


In [54]:
print("r2: %f" % trainingSummary.r2)

r2: 0.817863


### Predicting Maximum Salary

In [55]:
from pyspark.ml.regression import LinearRegression

# Create and train the Linear Regression model
lr1 = LinearRegression(featuresCol="features", labelCol="Max_sal", maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model_max = lr1.fit(train_data)

coefficients = lr_model_max.coefficients
intercept = lr_model_max.intercept

print("Coefficients for featuresCol: %s" % str(coefficients))
print("Intercept for labelCol: %s" % str(intercept))


Coefficients for featuresCol: [223735.59774663305,246998.61345516934,-16516.003531065366,-11331.842418701004,172381.10825678645,12414.179730244077,0.0,-212341.59038507706,347973.00565761584,126554.24654270813,-1885.166652955076,114273.32035995391,41100.73076807818,-113065.8517955153,-67823.10952629537,-83220.53829886431,73906.02197476939,0.0,-110539.88859292121,81455.50608666237,46604.35260871954,335162.7930320419,-95441.45249334216,102276.94370749143,284939.40588623215,-335547.63751548727,-180649.07232905529]
Intercept for labelCol: 100301.79321806037


In [58]:
trainingSummary = lr_model_min.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 172343.634616
r2: 0.817863


In [59]:
trainingSummary